# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [58]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [60]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [62]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [63]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [64]:
hourly_employees = salaries[salaries["Hourly Rate"].notna()]
hourly_employees

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
10,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,P,Hourly,20.0,NaN,2.65
18,"ABDULLAH, LAKENYA N",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
...,...,...,...,...,...,...,...,...
33164,"ZUREK, FRANCIS",ELECTRICAL MECHANIC,OEMC,F,Hourly,40.0,NaN,46.10
33168,"ZWARYCZ MANN, IRENE A",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
33169,"ZWARYCZ, THOMAS J",POOL MOTOR TRUCK DRIVER,WATER MGMNT,F,Hourly,40.0,NaN,35.60
33174,"ZYGADLO, JOHN P",MACHINIST (AUTOMOTIVE),GENERAL SERVICES,F,Hourly,40.0,NaN,46.35


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [66]:
from scipy.stats import ttest_1samp

# H0: mu = 30
# H1: mu =! 30
# alpha = 0,05 - 95% de confianza

mu = 30
alpha = 0.05

# Formula prueba de hipotesis
result = ttest_1samp(hourly_employees["Hourly Rate"], mu)
print(result)

# Interpretación del resultado
if result.pvalue < alpha:
    print("Rechazamos la hipótesis nula: el salario por hora es significativamente diferente de $30/hora.")
else:
    print("No se puede rechazar la hipótesis nula: no hay evidencia suficiente para afirmar que el salario por hora es diferente de $30/hora.")

TtestResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92, df=8021)
Rechazamos la hipótesis nula: el salario por hora es significativamente diferente de $30/hora.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [68]:
# Formula t.interval 
# alpha = 0.05 - 95% confianza
# grados libertad = nºfilas -1
# loc = mean 
# scale = std dev

confidence = 0.95

interval = t.interval(alpha=confidence, df=len(hourly_employees["Hourly Rate"])-1, loc=np.mean(hourly_employees["Hourly Rate"]), scale=sem(hourly_employees["Hourly Rate"]))

print(f" The hourly rate mean is in between the interval: {interval} with 95% confidence")


 The hourly rate mean is in between the interval: (32.52345834488425, 33.05365708767623) with 95% confidence


/var/folders/ws/nnfflmsx4qs43903148tzjnc0000gn/T/ipykernel_10897/3640224206.py:9: DeprecationWarning: Use of keyword argument 'alpha' for method 'interval' is deprecated and wil be removed in SciPy 1.11.0. Use first positional argument or keyword argument 'confidence' instead.
  interval = t.interval(alpha=confidence, df=len(hourly_employees["Hourly Rate"])-1, loc=np.mean(hourly_employees["Hourly Rate"]), scale=sem(hourly_employees["Hourly Rate"]))


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [70]:
#Importar la funcion proportions_ztest
from statsmodels.stats.proportion import proportions_ztest

# Parametros de la funcion
num_successes = len(hourly_employees["Hourly Rate"])
n = len(salaries["Hourly Rate"]) # la muestra es igual a los datos totales
p_null = 0.25 # proporción de H0

# Formula proportions_ztest
stat, p_value = proportions_ztest(count=num_successes, nobs=n, value=p_null)

# Resultados
print("Estadístico z:", stat)
print("Valor p:", p_value)

print("Nombre d'èxits (num_successes):", num_successes)
print("Mida de la mostra (n):", n)

Estadístico z: -3.5099964213703005
Valor p: 0.0004481127249057967
Nombre d'èxits (num_successes): 8022
Mida de la mostra (n): 33183


Com que el valor p és molt més petit que 0.05, podem rebutjar la hipòtesi nul·la. Això significa que tenim evidència suficient per concloure que la proporció de treballadors per hora a Chicago és significativament diferent del 25%. Concretament, el valor negatiu de l’estadístic z indica que la proporció observada és menor del 25%.